In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import json
import os
import numpy as np
import shutil # For sorting files

In [3]:
SORTED_DATA_DIR = '../data/user2/train_sorted'


IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR_USER2 = SORTED_DATA_DIR # Use the new sorted directory

datagen_u2 = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='training'
)
validation_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Very small cnn architecture due to computational limitations

model_v2 = Sequential([
    # Block 1 
    Conv2D(8, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2,2),

    # Block 2
    Conv2D(16, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Block 3
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    GlobalAveragePooling2D(),

    Dense(32, activation='relu'),
    Dropout(0.4),

    Dense(1, activation='sigmoid')
])


In [20]:
model_v2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 126, 126, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 63, 63, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 61, 61, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,121 (27.82 KB)

 Trainable params: 7,121 (27.82 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model_v2.compile(optimizer='adam',
                  loss='binary_crossentropy', # Changed for binary
                  metrics=['accuracy'])

In [22]:
history_v2 = model_v2.fit(train_generator_u2, epochs=5, validation_data=validation_generator_u2)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 172s 263ms/step - accuracy: 0.5736 - loss: 0.6743 - val_accuracy: 0.6076 - val_loss: 0.6556
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 190s 244ms/step - accuracy: 0.6202 - loss: 0.6436 - val_accuracy: 0.6448 - val_loss: 0.6300
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 149s 239ms/step - accuracy: 0.6623 - loss: 0.6149 - val_accuracy: 0.6924 - val_loss: 0.5908
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 159s 255ms/step - accuracy: 0.6794 - loss: 0.5993 - val_accuracy: 0.6988 - val_loss: 0.5804
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 148s 237ms/step - accuracy: 0.6973 - loss: 0.5846 - val_accuracy: 0.7162 - val_loss: 0.5633


In [23]:

# --- Print metrics before saving ---
print("\n===== TRAIN METRICS =====")
if "loss" in history_v2.history:
    print(f"Final Train Loss      : {history_v2.history['loss'][-1]:.4f}")
if "accuracy" in history_v2.history:
    print(f"Final Train Accuracy  : {history_v2.history['accuracy'][-1]:.4f}")

print("\n===== VALIDATION METRICS =====")
if "val_loss" in history_v2.history:
    print(f"Final Val Loss        : {history_v2.history['val_loss'][-1]:.4f}")
if "val_accuracy" in history_v2.history:
    print(f"Final Val Accuracy    : {history_v2.history['val_accuracy'][-1]:.4f}")



===== TRAIN METRICS =====
Final Train Loss      : 0.5846
Final Train Accuracy  : 0.6973

===== VALIDATION METRICS =====
Final Val Loss        : 0.5633
Final Val Accuracy    : 0.7162


In [25]:
# --- Ensure directories exist ---
os.makedirs("../models", exist_ok=True)
os.makedirs("../results", exist_ok=True)

# --- Save model ---
model_v2.save("../models/model_v2.h5")
print("\nSaved model_v2.h5 to models/")

# --- Save ALL metrics (not only last epoch) ---
metrics_v1_data = {}

for key, values in history_v2.history.items():
    # Convert numpy.float32 → Python float
    metrics_v1_data[key] = [float(v) for v in values]

# Save final epoch separately for convenience
metrics_v1_data["final"] = {
    "train_accuracy": float(history_v2.history["accuracy"][-1]) if "accuracy" in history_v2.history else None,
    "train_loss": float(history_v2.history["loss"][-1]) if "loss" in history_v2.history else None
  }

with open("../results/metrics_v2.json", "w") as f:
    json.dump(metrics_v1_data, f, indent=4)

print("Saved metrics_v2.json to results/")


Saved model_v2.h5 to models/
Saved metrics_v2.json to results/
